<a href="https://colab.research.google.com/github/wallace1588/tcc-dish-recommendation-rag/blob/main/notebooks/CHATBOT_2_ARENA_ANVIL_SERVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
BATTLE_NAME = 'BATTLE_NUM'
UP_LINK = 'ANVIL_LINK'

In [ ]:
!CMAKE_ARGS="-DGGML_CUDA=on" pip install pymongo anvil-uplink sentence-transformers langchain-community bitsandbytes flash_attn accelerate llama-cpp-python==0.2.88 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!curl -sSf https://raw.githubusercontent.com/WasmEdge/WasmEdge/master/utils/install.sh | bash -s -- -p /usr/local --plugins wasi_logging wasi_nn-ggml

Configuração para cpu: comente o import llama, a quantização, o tokenizer cuda

In [ ]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from pymongo.mongo_client import MongoClient
from huggingface_hub import hf_hub_download
from torch import bfloat16
import torch
import transformers
import urllib3
from bs4 import BeautifulSoup
from google.colab import userdata
import json
import os
import time
from llama_cpp import Llama
from multiprocessing import Process
import csv
import anvil.server

HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
N_TOKENS = 200
N_GPU_LAYERS = 120

class standard_model_eval:
  def __init__(self, model_id, retriever, flag_chat = 0):
    self.model_id = model_id
    self.flag_chat = flag_chat
    self.stop_strings =  ["user:", "Recbot:"]
    self.retriever = retriever
    self.tokenizer = None
    self.model = None

  def create(self):
    bnb_config = transformers.BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=bfloat16)
    self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_id, token=HF_TOKEN)
    self.model = transformers.AutoModelForCausalLM.from_pretrained(self.model_id, quantization_config=bnb_config, token=HF_TOKEN)

  def generate_text(self, prompt):
    inputs = self.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    outputs = self.model.generate(inputs, max_new_tokens = N_TOKENS, stop_strings = self.stop_strings, temperature=1.0, tokenizer = self.tokenizer, pad_token_id=self.tokenizer.eos_token_id)
    outputs =  outputs[0:,len(inputs[0]):]
    output_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

  def get_response(self, input_text):
    text = self.generate_text(self.prompt_gen(input_text))
    response = ''
    for stop_string in self.stop_strings:
      if text.find(stop_string) < 0:
        continue
      else:
        response = text[:text.find(stop_string)].strip('\n')
        break
    if response == '':
      response = text.strip('\n')
    return response

  def get_sugestions(self, input_text):
    THRESHOLD_MIN = 0.825
    list_docs = self.retriever.similarity_search_with_score(input_text, 3)
    intro_sugestions = 'Sugestões: '
    sugestions = intro_sugestions
    for r in list_docs:
      dish_description = r[0].page_content
      if r[1] > THRESHOLD_MIN:
        sugestions = sugestions + dish_description + '; '
    if sugestions == intro_sugestions:
      sugestions = 'Não há sugestões.'
    return sugestions

  def prompt_gen(self, user_input_list):
    prompt = ""
    prompt_shots = [
      {"role":"system","content":"Seu nome é Recbot e você deve fazer recomendações para auxiliar os usuários na escolha de refeições e restaurantes. Use as informações de entrada do usuário para gerar sua resposta. Caso não haja sugestões, você deve solicitar mais informações ao usuário ou responder baseado no histórico da conversa. Suas respostas devem conter apenas um linha."},
      {"role":"user","content":"Estou com fome.\nNão há sugestões."},
      {"role":"assistant","content":"Estou aqui para te ajudar. O que você gosta de comer?"},
      {"role":"user","content":"queria tomar guarana jesus.\nSugestões: Guarana Jesus: lata 350ml, 7.15, Pasta in Cup, 4.9, https://www.ifood.com.br/delivery/recife-pe/pasta-in-cup-torre/2d4543fd-cb1b-4cb8-b4c8-7c778dfacc0c, ifood; Refrigerante Guarana Jesus 350ml: lata 350ml, 7.0, Donico Coxinhas Artesanais Com Massa de, 4.9, https://www.ifood.com.br/delivery/recife-pe/donico-coxinhas-artesanais-com-massa-de-prado/ce9fffb6-23f4-4117-a604-9ff71ba78405, ifood; Refrigerante Jesus Guarana Lata 350ml: lata 350ml, 4.6, Padaria Ipiranga, 4.9, https://www.ifood.com.br/delivery/recife-pe/padaria-ipiranga-afogados/eb4b66e2-8469-46d5-acf0-b905326fe38f, ifood;"},
      {"role":"assistant","content":"Uma boa opção é o Guaraná Jesus 350ml da Padaria Ipiranga, por apenas R$ 4.60. Tamnbém tem no Pasta in Cup e no Donico Coxinhas Artesanais Com Massa de. Para mais informações acesse o link: https://www.ifood.com.br/delivery/recife-pe/padaria-ipiranga-afogados/eb4b66e2-8469-46d5-acf0-b905326fe38f."},
      {"role":"user","content":"Onde posso comprar?\nNão há sugestões."},
      {"role":"assistant","content":"Você pode comprar direto na Padaria Ipiranga ou através do site: https://www.ifood.com.br/delivery/recife-pe/padaria-ipiranga-afogados/eb4b66e2-8469-46d5-acf0-b905326fe38f."},
      {"role":"user","content":"Qual seu nome?\nNão há sugestões."},
      {"role":"assistant","content":"Meu nome é Recbot e estou que para te ajudar com as melhores dicas de restaurantes e pratos. Os mais gostosos da cidade"}
    ]
    prompt_shots_user = prompt_shots + user_input_list
    if self.flag_chat == 2:
      for user_input in prompt_shots_user:
        role = user_input['role']
        content = user_input['content']
        content = content.replace("'", "")
        content = content.replace('"', '')
        if role == 'user':
          prompt += ', {"role":"user", "content": "'
        elif role == 'system':
          prompt += '[{"role":"system", "content": "'
        else:
          prompt += ', {"role":"assistant", "content": "'
        prompt += content + '"}'
      prompt += ']'
      prompt = prompt.replace('\n', ' ')

    elif self.flag_chat == 1:
      prompt = self.tokenizer.apply_chat_template(prompt_shots_user, tokenize=False, add_generation_prompt=True, return_tensors="pt")
    else :
      for user_input in prompt_shots_user:
        role = user_input['role']
        content = user_input['content']
        if role == 'user':
          prompt += "user: {}\n".format(content)
        elif role == 'system':
          prompt += "{}\n\n".format(content)
        else:
          prompt += "Recbot: {}\n\n".format(content)
      prompt += "Recbot:"
    return prompt

class gguf_model(standard_model_eval):
  def __init__(self, model_id, retriever):
    super().__init__(model_id, retriever)

  def create(self):
    l_index = self.model_id.find('/') + 1
    n_index = self.model_id[l_index:].find('/')
    f_index = l_index + n_index
    model_language = self.model_id[:f_index]
    model_file = self.model_id[f_index + 1:]
    self.model = Llama.from_pretrained(repo_id = model_language, filename = model_file, verbose = False, n_ctx = 2048, n_gpu_layers = N_GPU_LAYERS)

  def generate_text(self, prompt):
    print(prompt)
    outputs = self.model(prompt, max_tokens = N_TOKENS, stop = self.stop_strings, temperature=1.0, echo = False)
    text = outputs['choices'][0]['text']
    return text

class bullerwins_model(standard_model_eval):
  def __init__(self, model_id, retriever):
    super().__init__(model_id, retriever)

  def create(self):
    hf_hub_download(repo_id="bullerwins/Meta-Llama-3.1-8B-Instruct-GGUF", filename="Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", revision="a4ac94cf28701b385c9028d49d314a361e0974a6", local_dir="/content")
    self.model = Llama(model_path="/content/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", verbose = False, n_batch = 256, n_ctx = 2048, n_gpu_layers = N_GPU_LAYERS)

  def generate_text(self, prompt):
    print(prompt)
    outputs = self.model(prompt, max_tokens = N_TOKENS, stop = self.stop_strings, temperature=1.0, echo = False)
    text = outputs['choices'][0]['text']
    return text

class second_state_model(standard_model_eval):
  def __init__(self, model_id, retriever):
    super().__init__(model_id, retriever, flag_chat = 2)

  def create(self):
    !{'curl -LO https://github.com/second-state/llamaedge/releases/latest/download/llama-api-server.wasm'}

    def start_server():
      l_index = self.model_id.find('/') + 1
      n_index = self.model_id[l_index:].find('/')
      f_index = l_index + n_index
      model_language = self.model_id[:f_index]
      model_file = self.model_id[f_index + 1:]
      c1 = f'curl -LO https://huggingface.co/{model_language}/resolve/main/{model_file}'
      c2 = f'wasmedge --dir .:. --nn-preload default:GGML:AUTO:{model_file} llama-api-server.wasm --prompt-template mistral-instruct --socket-addr 0.0.0.0:8000 --temp 1.0 --n-predict {N_TOKENS} --ctx-size 2048 --n-gpu-layers {N_GPU_LAYERS}'
      !{c1}
      !{c2}

    service = Process(target=start_server)
    service.start()
    while True:
      test_server =  !curl http://localhost:8000
      if test_server == []:
        break
      time.sleep(5)
      pass

  def generate_text(self, prompt):
    print(prompt)
    r = 'curl -X POST http://localhost:8000/v1/chat/completions -H \'accept:application/json\' -H \'Content-Type: application/json\' -d \'{"messages":' + prompt + '}\''
    response = !{r}
    json_response = json.loads(response[0])
    text = json_response['choices'][0]["message"]["content"]
    return text

In [ ]:
def get_retriever(retriver_model_name = "jmbrito/ptbr-similarity-e5-small"):
  MONGO_URI = "YOUR_URI"
  DATABASE_NAME = "YOUR_DB"
  COLLECTION_NAME = "YOUR_COLLECTION"
  ATLAS_VECTOR_SEARCH_INDEX_NAME = "YOUR_VECTOR_INDEX"
  mongo_client = MongoClient(MONGO_URI)
  mycol = mongo_client[DATABASE_NAME][COLLECTION_NAME]
  embeddings = HuggingFaceEmbeddings(model_name=retriver_model_name)
  vectore_store = MongoDBAtlasVectorSearch(collection= mycol, embedding=embeddings, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME)
  return vectore_store

In [ ]:
def get_model(model_name, retriever_model):
  if 'second-state' in model_name:
    return second_state_model(model_name, retriever_model)
  elif 'bullerwins' in model_name:
    return bullerwins_model(model_name, retriever_model)
  elif '.gguf' in model_name:
    return gguf_model(model_name, retriever_model)
  url_hugging = 'https://huggingface.co/'
  response = urllib3.request('GET', 'https://huggingface.co/{}'.format(model_name))
  html_doc =  response.data.decode('utf-8')
  soup = BeautifulSoup(html_doc, 'html.parser')
  list_a = soup.find_all("a", {"class": "mb-1 mr-1 md:mb-1.5 md:mr-1.5 rounded-lg"}, href=True)
  list_names_language = ['bloom','falcon','xglm','mistral','stablelm','gpt2', 't5','llama','nvidia']

  for a in list_a:
    text_a = a.getText().strip('\n')
    if text_a in list_names_language:
      return standard_model_eval(model_name, retriever_model)

  return standard_model_eval(model_name, retriever_model, 1)

In [ ]:
battles_chatbots = {
    'battle 01': ['second-state/Mistral-Nemo-Instruct-2407-GGUF/Mistral-Nemo-Instruct-2407-Q5_0.gguf','bullerwins/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf'],
    'battle 02': ['lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf','bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf'],
    'battle 03': ['MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct.Q5_K_M.gguf','second-state/Mistral-Nemo-Instruct-2407-GGUF/Mistral-Nemo-Instruct-2407-Q5_0.gguf'],
    'battle 04': ['bullerwins/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf','lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf'],
    'battle 05': ['bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf','MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct.Q5_K_M.gguf'],
    'battle 06': ['second-state/Mistral-Nemo-Instruct-2407-GGUF/Mistral-Nemo-Instruct-2407-Q5_0.gguf','lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf'],
    'battle 07': ['bullerwins/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf','MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct.Q5_K_M.gguf'],
    'battle 08': ['bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf','second-state/Mistral-Nemo-Instruct-2407-GGUF/Mistral-Nemo-Instruct-2407-Q5_0.gguf'],
    'battle 09': ['lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf','MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct.Q5_K_M.gguf'],
    'battle 10': ['bullerwins/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf','bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-IQ4_XS.gguf']
}

In [ ]:
battle_atual = battles_chatbots[BATTLE_NAME]
retriever = get_retriever()
model = get_model(battle_atual[1], retriever)
model.create()
torch.cuda.empty_cache()
anvil.server.connect(UP_LINK)
name_file = 'chatbot_arena_chat2_{}.csv'.format(BATTLE_NAME)
with open(name_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['USER','CHATBOT 2'])

In [ ]:
sessions = {}

@anvil.server.callable
def get_conversation_2():
  #session_id = anvil.server.get_session_id()
  session_id = 'chat_2'
  if session_id not in sessions:
    sessions[session_id] = {}
    session = sessions[session_id]
  else:
    session = sessions[session_id]
  chatBot_id = 'chatbot 2'
  if chatBot_id not in session:
    session[chatBot_id] = []
  return session[chatBot_id]

In [ ]:
@anvil.server.callable
def set_conversation_2(chat):
  #session_id = anvil.server.get_session_id()
  session_id = 'chat_2'
  session = sessions[session_id]
  chatBot_id = 'chatbot 2'
  session[chatBot_id] = chat

In [ ]:
@anvil.server.background_task
def get_model_response_2(text):
  chat_2 = get_conversation_2()
  input_user = text + '\n' + model.get_sugestions(text)
  if len(chat_2) > 4:
    chat_2 = chat_2[-4:]
    get_conversation_2().append({"role": "user", "content": input_user})
  chat_2.append({"role": "user", "content": input_user})
  model_2_response = model.get_response(chat_2)
  with open(name_file, 'a') as f:
    writer = csv.writer(f)
    writer.writerow([text, model_2_response])
  get_conversation_2().append({"role": "assistant", "content": model_2_response})

In [ ]:
num_chat_1 = 0
num_chat_2 = 0

@anvil.server.callable
def set_result_2(chat_id):
  with open(name_file, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['----------------', '----------------'])

  global num_chat_1
  global num_chat_2
  if chat_id == 'Chatbot 1':
    num_chat_1 += 1
  else:
    num_chat_2 += 1

  os.system('cls')
  print(f'{battle_atual[0]}: {num_chat_1}\n{battle_atual[1]}: {num_chat_2}')
  sessions['chat_2']['chatbot 2'] = []

In [ ]:
@anvil.server.callable
def close_server_2():
  anvil.server.disconnect()

In [ ]:
anvil.server.wait_forever()